In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.nasnet import NASNetMobile
import cv2
import math
import os
from glob import glob
from scipy import stats as s
from sklearn.externals import joblib 

Using TensorFlow backend.
C:\Users\HH TRADERS\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
base_model = NASNetMobile(include_top=False, weights='imagenet')

In [3]:
base_model.summary()

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           stem_bn1[0][0]                   
_____________________________________________________________________________________________

separable_conv_2_reduction_righ (None, 56, 56, 11)   396         activation_9[0][0]               
__________________________________________________________________________________________________
separable_conv_2_reduction_left (None, 56, 56, 11)   220         activation_11[0][0]              
__________________________________________________________________________________________________
adjust_relu_1_stem_2 (Activatio (None, 111, 111, 32) 0           stem_bn1[0][0]                   
__________________________________________________________________________________________________
reduction_add_2_stem_1 (Add)    (None, 56, 56, 11)   0           reduction_left2_stem_1[0][0]     
                                                                 separable_conv_2_bn_reduction_rig
__________________________________________________________________________________________________
reduction_left3_stem_1 (Average (None, 56, 56, 11)   0           reduction_pad_1_stem_1[0][0]     
__________

separable_conv_2_bn_reduction_l (None, 28, 28, 22)   88          separable_conv_2_reduction_left1_
__________________________________________________________________________________________________
separable_conv_2_bn_reduction_r (None, 28, 28, 22)   88          separable_conv_2_reduction_right1
__________________________________________________________________________________________________
separable_conv_1_pad_reduction_ (None, 61, 61, 22)   0           activation_17[0][0]              
__________________________________________________________________________________________________
activation_19 (Activation)      (None, 56, 56, 22)   0           adjust_bn_stem_2[0][0]           
__________________________________________________________________________________________________
reduction_add_1_stem_2 (Add)    (None, 28, 28, 22)   0           separable_conv_2_bn_reduction_lef
                                                                 separable_conv_2_bn_reduction_rig
__________

adjust_conv_1_0 (Conv2D)        (None, 28, 28, 22)   968         adjust_avg_pool_1_0[0][0]        
__________________________________________________________________________________________________
adjust_conv_2_0 (Conv2D)        (None, 28, 28, 22)   968         adjust_avg_pool_2_0[0][0]        
__________________________________________________________________________________________________
activation_23 (Activation)      (None, 28, 28, 88)   0           reduction_concat_stem_2[0][0]    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 28, 28, 44)   0           adjust_conv_1_0[0][0]            
                                                                 adjust_conv_2_0[0][0]            
__________________________________________________________________________________________________
normal_conv_1_0 (Conv2D)        (None, 28, 28, 44)   3872        activation_23[0][0]              
__________

                                                                 separable_conv_2_bn_normal_right1
__________________________________________________________________________________________________
normal_add_2_0 (Add)            (None, 28, 28, 44)   0           separable_conv_2_bn_normal_left2_
                                                                 separable_conv_2_bn_normal_right2
__________________________________________________________________________________________________
normal_add_3_0 (Add)            (None, 28, 28, 44)   0           normal_left3_0[0][0]             
                                                                 adjust_bn_0[0][0]                
__________________________________________________________________________________________________
normal_add_4_0 (Add)            (None, 28, 28, 44)   0           normal_left4_0[0][0]             
                                                                 normal_right4_0[0][0]            
__________

...

__________________________________________________________________________________________________
activation_148 (Activation)     (None, 7, 7, 176)    0           adjust_bn_9[0][0]                
__________________________________________________________________________________________________
activation_150 (Activation)     (None, 7, 7, 176)    0           normal_bn_1_9[0][0]              
__________________________________________________________________________________________________
separable_conv_1_normal_left1_9 (None, 7, 7, 176)    35376       activation_142[0][0]             
__________________________________________________________________________________________________
separable_conv_1_normal_right1_ (None, 7, 7, 176)    32560       activation_144[0][0]             
__________________________________________________________________________________________________
separable_conv_1_normal_left2_9 (None, 7, 7, 176)    35376       activation_146[0][0]             
__________

__________________________________________________________________________________________________
activation_152 (Activation)     (None, 7, 7, 704)    0           reduction_concat_reduce_8[0][0]  
__________________________________________________________________________________________________
activation_153 (Activation)     (None, 7, 7, 1056)   0           normal_concat_9[0][0]            
__________________________________________________________________________________________________
adjust_conv_projection_10 (Conv (None, 7, 7, 176)    123904      activation_152[0][0]             
__________________________________________________________________________________________________
normal_conv_1_10 (Conv2D)       (None, 7, 7, 176)    185856      activation_153[0][0]             
__________________________________________________________________________________________________
adjust_bn_10 (BatchNormalizatio (None, 7, 7, 176)    704         adjust_conv_projection_10[0][0]  
__________

normal_add_2_10 (Add)           (None, 7, 7, 176)    0           separable_conv_2_bn_normal_left2_
                                                                 separable_conv_2_bn_normal_right2
__________________________________________________________________________________________________
normal_add_3_10 (Add)           (None, 7, 7, 176)    0           normal_left3_10[0][0]            
                                                                 adjust_bn_10[0][0]               
__________________________________________________________________________________________________
normal_add_4_10 (Add)           (None, 7, 7, 176)    0           normal_left4_10[0][0]            
                                                                 normal_right4_10[0][0]           
__________________________________________________________________________________________________
normal_add_5_10 (Add)           (None, 7, 7, 176)    0           separable_conv_2_bn_normal_left5_
          

__________________________________________________________________________________________________
separable_conv_2_bn_normal_righ (None, 7, 7, 176)    704         separable_conv_2_normal_right1_11
__________________________________________________________________________________________________
separable_conv_2_bn_normal_left (None, 7, 7, 176)    704         separable_conv_2_normal_left2_11[
__________________________________________________________________________________________________
separable_conv_2_bn_normal_righ (None, 7, 7, 176)    704         separable_conv_2_normal_right2_11
__________________________________________________________________________________________________
normal_left3_11 (AveragePooling (None, 7, 7, 176)    0           normal_bn_1_11[0][0]             
__________________________________________________________________________________________________
normal_left4_11 (AveragePooling (None, 7, 7, 176)    0           adjust_bn_11[0][0]               
__________

__________________________________________________________________________________________________
activation_183 (Activation)     (None, 7, 7, 176)    0           separable_conv_1_bn_normal_left2_
__________________________________________________________________________________________________
activation_185 (Activation)     (None, 7, 7, 176)    0           separable_conv_1_bn_normal_right2
__________________________________________________________________________________________________
activation_187 (Activation)     (None, 7, 7, 176)    0           separable_conv_1_bn_normal_left5_
__________________________________________________________________________________________________
separable_conv_2_normal_left1_1 (None, 7, 7, 176)    35376       activation_179[0][0]             
__________________________________________________________________________________________________
separable_conv_2_normal_right1_ (None, 7, 7, 176)    32560       activation_181[0][0]             
__________

In [4]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(51744,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(51, activation='softmax'))

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              52986880  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

In [7]:
# loading the trained weights
model.load_weights("../Models/weightNASNetMobile.hdf5")

In [8]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [9]:
location='../data/test'
classes=[]
videos=[]
test=pd.DataFrame()
for foldername in sorted(os.listdir(location)):
    classes.append(foldername)
    for files in sorted(os.listdir(location+'/'+foldername)):
        videos.append(foldername+'/'+files)
test['video_name']=videos
print(classes)
test_videos=test['video_name']
test.head()

['brush_hair', 'cartwheel', 'catch', 'chew', 'clap', 'climb', 'climb_stairs', 'dive', 'draw_sword', 'dribble', 'drink', 'eat', 'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'hit', 'hug', 'jump', 'kick', 'kick_ball', 'kiss', 'laugh', 'pick', 'pour', 'pullup', 'punch', 'push', 'pushup', 'ride_bike', 'ride_horse', 'run', 'shake_hands', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault', 'stand', 'swing_baseball', 'sword', 'sword_exercise', 'talk', 'throw', 'turn', 'walk', 'wave']


,video_name
0,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
1,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
2,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
3,brush_hair/Blonde_being_brushed_brush_hair_f_n...
4,brush_hair/Blonde_being_brushed_brush_hair_u_c...


In [10]:
# creating the tags
train = pd.read_csv('../data/train.csv')
y = train['class']
y = pd.get_dummies(y)

In [11]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    #print('../data/test/'+videoFile)
    cap = cv2.VideoCapture('../data/test/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('../data/temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
#            print('../data/temp/' + "_frame%d.jpg" % count)
            filename ='../data/temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("../data/temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*1056)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[0])


100%|██████████| 1530/1530 [24:43<00:00,  1.31it/s] 


In [12]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

20.19607843137255

In [13]:
from sklearn import metrics
print(metrics.classification_report(actual,predict))

C:\Users\HH TRADERS\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                precision    recall  f1-score   support

    brush_hair       0.35      0.30      0.32        30
     cartwheel       0.07      0.10      0.08        30
         catch       0.00      0.00      0.00        30
          chew       0.41      0.23      0.30        30
          clap       0.21      0.20      0.21        30
         climb       0.75      0.50      0.60        30
  climb_stairs       0.00      0.00      0.00        30
          dive       0.26      0.20      0.23        30
    draw_sword       0.13      0.17      0.14        30
       dribble       0.22      0.73      0.34        30
         drink       0.11      0.37      0.17        30
           eat       0.16      0.10      0.12        30
    fall_floor       0.00      0.00      0.00        30
       fencing       0.00      0.00      0.00        30
     flic_flac       0.17      0.10      0.12        30
          golf       0.80      0.27      0.40        30
     handstand       0.11      0.57      0.18  

In [14]:
cm=metrics.confusion_matrix(actual,predict)
print(cm)

[[ 9  0  0 ...  0 11  0]
 [ 0  3  0 ...  0  0  0]
 [ 0  3  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  2 14  0]
 [ 0  0  0 ...  0 22  1]
 [ 1  0  0 ...  0  9  1]]


In [15]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square=True, cmap="Blues_r");
plt.ylabel("Actual label");
plt.xlabel("Predicted label")
all_sample_title="Accuracy score: {0}".format(metrics.accuracy_score(predict, actual)*100)
plt.title(all_sample_title,size=15)

Text(0.5, 1, 'Accuracy score: 20.19607843137255')